In [3]:
# Import Libraries
import pandas as pd
import math
import numpy as np
import sklearn
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers

from sklearn import preprocessing

from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

from sklearn.svm import SVR
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.linear_model import LinearRegression, LogisticRegression

import matplotlib.pyplot as plt

In [4]:
# Data loading
data = pd.read_csv('/content/ISCTR.IS (1).csv', parse_dates=True, index_col=0)

data = data.dropna()

data['High_Low_per'] = (data['High'] - data['Close']) / data['Close'] * 100
data['Per_change'] = (data['Open'] - data['Open']) / data['Close'] * 100
data = data[['Adj Close', 'High_Low_per', 'Per_change', 'Volume']]
label_col = 'Adj Close'

forecast_ceil = int(math.ceil(0.002*len(data)))
data['label'] = data[label_col].shift(-forecast_ceil)

# Drop the label column and convert to numpy array
X = data.drop(['label'], axis=1).values
X = preprocessing.scale(X)
X = X[:-forecast_ceil]
X_lately = X[-forecast_ceil:]

# Drop rows with missing values
data.dropna(inplace=True)

# Convert label column to numpy array
y = data['label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

<ipython-input-4-c7d49bb74d2a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'] = data[label_col].shift(-forecast_ceil)
<ipython-input-4-c7d49bb74d2a>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [5]:
svr = SVR(kernel='rbf', C=1000, gamma=0.1)
svr.fit(X_train, y_train)

# Predict the test data
prediction_svm = svr.predict(X_test)

# Predict the next 7 days of data
svm_next7days = svr.predict(X_lately)

print('Predictions on Test Data:', prediction_svm)
print('Predictions for Next 7 Days:', svm_next7days)

Predictions on Test Data: [ 2.48153668  4.68703237  2.74682604  1.84059892  3.735609    2.90716939
  2.05514527  2.25407828  2.06921861  1.76375567  1.78186765  1.95925071
  1.90721476  2.05512504  2.45688767  2.27947382  1.80528992  2.77754862
  2.06725802  2.2978898   2.19319513  1.91407628  4.60664986  1.77384292
  1.7184287   2.46434933  1.93468462  2.72046745  1.90029466  2.37247716
  2.89245093  2.51801646  2.6655631   2.11895872  1.88165686  1.82376506
  2.13810445  2.60655582  2.8632394   2.37956279  1.90715989  3.4432323
  2.5246812   2.83149969  3.04962675  2.25244061  2.18922632  2.12833724
  1.81949102  1.73719761  1.7280617   2.21164098  2.61721378  2.63351222
 13.30518963  2.76506103  4.30268026  2.86435818  2.8020468   2.19645141
  2.52425613  2.47433599  4.9218001   2.10173023  2.25917761  1.90098836
  2.27088253  1.84348841  2.40288635  3.20188644  3.40708963  1.83738847
  2.03347702  9.15195714  3.17316452  2.71366359 11.89296566  2.7491424
  1.70877782  2.71432679  2

In [6]:
svm_accuracy = svr.score(X_test, y_test)

print(svm_accuracy*100)

95.55228854173333


In [7]:
lr = LinearRegression()
lr.fit(X_train, y_train)

lr_prediction = lr.predict(X_test)

# Predict the next 7 days of data
lr_next7days = lr.predict(X_lately)

print('Predictions on Test Data:', lr_prediction)
print('Predictions for Next 7 Days:', lr_next7days)

Predictions on Test Data: [ 2.48021327  4.53401676  2.74373058  1.80931396  3.79727646  2.92929452
  4.0512449   2.27874049  2.0506782   1.74539442  1.76968249  1.93541137
  1.89179864  2.0666789   2.48025589  2.31874235  1.82706966  2.80510461
  2.04782752  7.7420904   2.23048273  1.93202327  4.63218723  1.78645397
  1.73322029  2.46481967  1.9576942   2.73299432  1.93613004  2.36533861
  2.92730338  2.56705678  2.68141111  2.10596179  1.85881241  1.87968346
  2.16474668  2.62480223  2.88203682  2.44746838  1.94454559  3.47928748
  2.52590804  2.85598461  3.05454717  2.25065786  2.2162436   2.14733528
  1.81425289  1.75578854  1.71882546  2.2125829   2.63046198  2.6449633
 12.99592934  2.77788637  4.3775944   2.88083921  2.81524491  2.2239564
  2.53966966  2.4779694   4.63556392  2.09321208  2.30491327  1.89590272
  2.3262273   1.8047915   2.40304268  3.30626922  3.60020082  1.84159542
  2.00189345  9.39212282  3.27238621  2.7343887  13.06033341  2.76584713
  1.76857978  2.73159964  2

In [8]:
lr_accuracy = lr.score(X_test, y_test)

print(lr_accuracy*100)

98.24569760198318


In [9]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs = 40, batch_size = 30, validation_data = (X_test, y_test))

Epoch 1/40
55/55 [==============================] - 9s 43ms/step - loss: 6.8862 - val_loss: 0.9485
Epoch 2/40
55/55 [==============================] - 1s 12ms/step - loss: 1.0572 - val_loss: 0.6649
Epoch 3/40
55/55 [==============================] - 1s 13ms/step - loss: 0.8590 - val_loss: 0.5161
Epoch 4/40
55/55 [==============================] - 1s 13ms/step - loss: 0.6832 - val_loss: 0.4065
Epoch 5/40
55/55 [==============================] - 1s 13ms/step - loss: 0.5579 - val_loss: 0.3266
Epoch 6/40
55/55 [==============================] - 1s 13ms/step - loss: 0.4739 - val_loss: 0.2518
Epoch 7/40
55/55 [==============================] - 1s 13ms/step - loss: 0.3789 - val_loss: 0.2068
Epoch 8/40
55/55 [==============================] - 1s 13ms/step - loss: 0.3406 - val_loss: 0.1861
Epoch 9/40
55/55 [==============================] - 1s 13ms/step - loss: 0.3492 - val_loss: 0.1936
Epoch 10/40
55/55 [==============================] - 1s 13ms/step - loss: 0.3012 - val_loss: 0.1540
Epoch 11/

In [11]:
dnn_prediction = model.predict(X_test)

# Predict for the next 7 days data
dnn_next7days = model.predict(X_lately)

print('Predictions on Test Data:', dnn_prediction)
print('Predictions for Next 7 Days:', dnn_next7days)

1/1 [==============================] - 0s 34ms/step
Predictions on Test Data: [[ 2.4632854]
 [ 4.6672716]
 [ 2.7556663]
 [ 1.9344596]
 [ 3.6496491]
 [ 2.7747042]
 [ 3.5265992]
 [ 2.1496577]
 [ 2.0876844]
 [ 1.8441117]
 [ 1.8291229]
 [ 2.050564 ]
 [ 1.9384813]
 [ 2.0455754]
 [ 2.3366857]
 [ 2.1575067]
 [ 1.7305237]
 [ 2.6684446]
 [ 2.0929525]
 [ 8.971633 ]
 [ 2.0907743]
 [ 1.8262813]
 [ 4.5850234]
 [ 1.7962389]
 [ 1.7108128]
 [ 2.4841259]
 [ 1.9236729]
 [ 2.7092032]
 [ 1.9016104]
 [ 2.3430777]
 [ 2.9396245]
 [ 2.3657658]
 [ 2.6373303]
 [ 2.1282065]
 [ 2.0253508]
 [ 1.8682648]
 [ 2.0512702]
 [ 2.5662384]
 [ 2.7269003]
 [ 2.31264  ]
 [ 1.8694168]
 [ 3.4143784]
 [ 2.3333306]
 [ 2.722604 ]
 [ 2.9751258]
 [ 2.2267427]
 [ 2.0601718]
 [ 2.0494955]
 [ 1.7397956]
 [ 1.7289515]
 [ 1.8155537]
 [ 2.1862407]
 [ 2.6386955]
 [ 2.6159534]
 [11.248659 ]
 [ 2.7286985]
 [ 4.312081 ]
 [ 2.7263024]
 [ 2.6958115]
 [ 2.1288135]
 [ 2.5694396]
 [ 2.435821 ]
 [ 4.828223 ]
 [ 2.102524 ]
 [ 2.1230078]
 [ 1.8959194